In [1]:
import numpy as np
import pandas as pd
import csv
import datetime
from datapathfile import path_lookup_dir, datafile
import re
from geopy import geocoders
import tzwhere

In [2]:
datafile

'rtb_data 2.csv'

In [3]:
def listcolumn(x):
    x = x.translate(None, '"')
    x = x.translate(None, ' ')
    x = x[1:-1].split(",")
    j = x[-1]
    return j

In [4]:
def TrueFalse(value):
    if value[0]=='F':
        value = False
    else:
        value = True
    return value

In [5]:
def Model(device):
    if device[0]=='F':
        value = False
    else:
        value = True
    return value

In [6]:
filename='ATA2_data.csv'

In [7]:
reader = list(csv.reader(open(path_lookup_dir+datafile, 'rb'), delimiter='\t'))
colheaders = reader[0] 
df = pd.DataFrame(reader[1:])
dffilter = df.iloc[:, 0:20]
dffilter.columns = [colheaders]

In [8]:
dffilter['device'] = dffilter['model']

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
rows_phone = dffilter[dffilter['device'].str.match('iPhone')]['device'].index
rows_music = dffilter[dffilter['device'].str.match('iPod')]['device'].index
rows_tablet = dffilter[dffilter['device'].str.match('iPad')]['device'].index


In [10]:
dffilter.ix[rows_phone, 20 ] = 'Phone'


/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
dffilter.ix[rows_music, 20] = 'Music Player'
dffilter.ix[rows_tablet, 20] = 'Tablet'

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
df = dffilter

In [13]:
df['impression'] = df['impression'].apply(TrueFalse)

In [14]:
df['installed'] = df['installed'].apply(TrueFalse)

In [15]:
df['clicked'] = df['clicked'].apply(TrueFalse)

In [16]:
df.ix[:, 'datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")
df.ix[:, 'app_type'] = df['app_categories'].apply(listcolumn)


In [30]:
df['weekday'].unique()

array([6, 7, 1])

In [18]:
drop_index=df[np.logical_and(
        np.logical_and(df['impression'], df['installed']),
        ~ df['clicked'])].index

In [19]:
empty_index = np.where(df['app_type'] == '')
ul_index = np.where(df['app_type'] == 'ul')
df = df.drop(df.index[empty_index])
df = df.drop(df.index[ul_index])
df = df.drop(df.index[drop_index])

In [20]:
df['weekday'] = df['datetime'].map(lambda x: x.isoweekday())
df['hours'] = df['datetime'].map(lambda x: x.hour)
df.ix[:, 'week'] = df['datetime'].map(lambda x:x.isocalendar()[1])

In [21]:
g = geocoders.GoogleV3()
place, (lat,lng) = g.geocode('Saint-felicien')

print place, (lat,lng)
timezone = g.timezone((lat,lng))
print timezone

Saint-Félicien, France (45.086423, 4.627967)
Europe/Paris


In [22]:
df['country'].unique()

array(['USA', 'CAN'], dtype=object)

In [23]:
weird_cities = ['Qu', 'Montr', 'Trois', 'cancour', 'Saint']
normal_cities = ['Quebec', 'Montreal', 'Trois-Rivieres','becancour', 'saint-felicien']

for i,index in enumerate(weird_cities):
    rows = df[df['city'].str.match(weird_cities[i])]['city'].index
    df.ix[rows, 15 ] = normal_cities[i]


In [24]:
impressiondf = df.groupby('city')['impression','clicked'].sum()
impression_index = impressiondf[impressiondf['impression'] > 5].index

In [25]:
df=df[df.city.isin(impression_index)]

In [26]:
df['location'] = df.city+','+ df.country



In [27]:
df

,datetime,advertiser_bundle_id,creative_id,ad_id,app_id,bidadop_id,app_name,app_categories,publisher_name,device_idfa,...,language,impression,clicked,installed,device,app_type,weekday,hours,week,location
0,2015-05-16 05:36:08,4,1199274520,1136359960,61569562,10273094,iOS Word Search,"[""IAB1"", ""IAB9"", ""IAB9-30"", ""entertainment"", ""...",Kevin Tydlacka,caf7bc3bf37f31e1032c1f11dafe1e90a9716676,...,en,True,False,False,Phone,games,6,5,20,"Springfield,USA"
1,2015-05-17 23:47:22,4,1196128792,1136359960,2011920920,10275318,BHUG103 - Monster Dentistâ¢,"[""IAB9"", ""IAB9-30"", ""games""]",Bear Hug Media,d1d8f0a8f41ed12b669d0c8a5e99d14d8627d71f,...,en,True,False,False,Phone,games,7,23,20,"Crystal Lake,USA"
2,2015-05-17 03:07:39,4,1202420248,1136359960,512457247,10276243,CSR Racing iOS,"[""IAB1"", ""IAB9"", ""IAB9-30"", ""entertainment"", ""...",NaturalMotion,8de62f21453ca921c0922d69f06c589dfda93aa8,...,fr,True,False,False,Phone,games,7,3,20,"Montreal,CAN"
3,2015-05-16 17:26:28,4,1204517400,1136359960,1106999837,10277334,RU_BorbaUmov_iOS,"[""IAB9"", ""IAB9-30"", ""games""]",FEO Media AB,314ffdc7828040f143e3f6225041ac94e84af103,...,ru,True,False,False,Phone,games,6,17,20,"markham,CAN"
4,2015-05-16 05:40:33,4,1198225944,1136359960,1314617882,10277892,Watch & Listen,"[""IAB1"", ""IAB1-6"", ""entertainment"", ""music""]",IAC Search & Media Europe Ltd,475fa916ffebf05c04cccbae41a22a09677a8e5a,...,en,True,False,False,Phone,music,6,5,20,"Sydney,CAN"
5,2015-05-16 23:43:19,4,1204517400,1136359960,1765505554,10278143,Call of Doodie - iOS,"[""IAB1"", ""IAB9"", ""IAB9-30"", ""entertainment"", ""...",Tapps,f96d2fb7d1be5f0f36481648b916a9622b4263c8,...,en,True,False,False,Phone,games,6,23,20,"Boston,USA"
6,2015-05-16 13:41:56,4,1204517400,1136359960,2092661272,10279304,Bubble Galaxy with Buddies iPhone,"[""IAB1"", ""IAB9"", ""IAB9-30"", ""entertainment"", ""...","Scopely, Inc.",61af0b234ccd41f3508e61acaa99c8e3147cc872,...,en,True,False,False,Phone,games,6,13,20,"Williamstown,USA"
7,2015-05-17 05:22:04,4,1196128792,1136359960,1255897624,10280198,Ruzzle iOS,"[""IAB9"", ""IAB9-30"", ""games""]",Mobile Access Group Interactive Sweden AB,650a7a44083ffb643303938eddb20b8c978b5d76,...,en,True,False,False,Phone,games,7,5,20,"El Paso,USA"
8,2015-05-16 20:40:57,4,1199274520,1136359960,1325103647,10280455,Cribbage Pro,"[""IAB9"", ""IAB9-30"", ""games""]","Fuller Systems, Inc.",e461d647465f70b5c02cfb24e065649db1da31a7,...,en,True,False,False,Phone,games,6,20,20,"Altamonte Springs,USA"
9,2015-05-17 13:53:29,4,1199274520,1136359960,731609627,10281837,"SNIPER306 - Guides for Pokemon XY:Videos,Walkt...","[""IAB9"", ""IAB9-30"", ""games""]",Sniper Studio,d374c5e992afa0e535718e0d14e1f74405039a4c,...,en,True,False,False,Music Player,games,7,13,20,"Shoreham,USA"


In [28]:
df.to_csv(filename)

In [29]:
filename

'ATA2_data.csv'